In [1]:
from typing import Dict

from prefect import Flow, task, case
from prefect import Parameter

from lume_services.results import Result
from lume_services.tasks import (
    configure_lume_services,
    prepare_lume_model_variables,
    check_local_execution,
    SaveDBResult,
    LoadDBResult,
    LoadFile,
    SaveFile,
)
from lume_services.files import TextFile
from lume_model.variables import InputVariable, OutputVariable
from prefect.storage import Module

from lume_lcls_cu_inj_nn.model import LCLSCuInjNN
from lume_lcls_cu_inj_nn import INPUT_VARIABLES, CU_INJ_MAPPING_TABLE

2023-08-01 21:30:13.684000: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def preprocessing_task(input_variables):
    # scale all values w.r.t. impact factor
    for var_name, var in input_variables.items():

        # if name included in scaling factors
        if (
            CU_INJ_MAPPING_TABLE["impact_name"]
            .str.contains(var_name, regex=False)
            .any()
        ):

            # scaled_val = (
            #     var.value
            #     * CU_INJ_MAPPING_TABLE.loc[
            #         CU_INJ_MAPPING_TABLE["impact_name"] == var_name, "impact_factor"
            #     ].item()
            # )

            var.value = var.value

    return input_variables

def format_result(
    input_variables: Dict[str, InputVariable],
    output_variables: Dict[str, OutputVariable],
):

    inputs = {var_name: var.value for var_name, var in input_variables.items()}
    outputs = {var_name: var.value.astype('float64') for var_name, var in output_variables.items()}

    # convert array to list
    outputs["x:y"] = outputs["x:y"].tolist()

    return Result(inputs=inputs, outputs=outputs)

In [3]:
configure = configure_lume_services.run()
running_local = check_local_execution.run()
#running_local.set_upstream(configure)

In [ ]:
input = {
                        "distgen:r_dist:sigma_xy:value": 0.45330, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
}

In [4]:
input_variable_parameter_dict = {
        var_name: Parameter(var_name, default=var.default)
        for var_name, var in INPUT_VARIABLES.items()
    }
INPUT_VARIABLES.keys()

dict_keys(['distgen:r_dist:sigma_xy:value', 'distgen:t_dist:length:value', 'distgen:total_charge:value', 'SOL1:solenoid_field_scale', 'CQ01:b1_gradient', 'SQ01:b1_gradient', 'L0A_phase:dtheta0_deg', 'L0A_scale:voltage', 'end_mean_z'])

In [5]:
# ORGANIZE INPUT VARIABLE VALUES LUME-MODEL VARIABLES
formatted_input_vars = prepare_lume_model_variables.run(
    input_variable_parameter_dict, INPUT_VARIABLES
)
processed_input_vars = preprocessing_task(formatted_input_vars)
model = LCLSCuInjNN()

2023-08-01 21:30:19.306272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-01 21:30:19.306488: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-01 21:30:19.324270: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
input = {
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
}

In [7]:
for key in formatted_input_vars:
    if key in input :
        formatted_input_vars[key].value = input[key]
    else:
        formatted_input_vars[key].value = 0
formatted_input_vars

{'distgen:r_dist:sigma_xy:value': ScalarInputVariable(variable_type='scalar', units='mm', parent_variable=None, value_range=[0.2000128411163704, 0.499960029319067], name='distgen:r_dist:sigma_xy:value', value=0.413, precision=None, default=0.413, is_constant=False),
 'distgen:t_dist:length:value': ScalarInputVariable(variable_type='scalar', units='ps', parent_variable=None, value_range=[3.0000935388576626, 11.999451344364768], name='distgen:t_dist:length:value', value=7.499772441611215, precision=None, default=7.499772441611215, is_constant=False),
 'distgen:total_charge:value': ScalarInputVariable(variable_type='scalar', units='pC', parent_variable=None, value_range=[250.0, 250.0], name='distgen:total_charge:value', value=250.0, precision=None, default=250.0, is_constant=True),
 'SOL1:solenoid_field_scale': ScalarInputVariable(variable_type='scalar', units='T', parent_variable=None, value_range=[0.15000051301097236, 0.29998391898360177], name='SOL1:solenoid_field_scale', value=0.17, p

In [8]:
output_variables = model.evaluate(formatted_input_vars)
formatted_input_vars

[array([[ 4.13000000e-01,  7.49977244e+00,  2.50000000e+02,
         1.70000000e-01, -7.40000000e-03, -7.40000000e-03,
        -8.89970000e+00,  7.00000000e+07,  4.61470020e+00]])]
1/1 [==============================] - 1s 724ms/step


2023-08-01 21:30:27.310318: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


{'distgen:r_dist:sigma_xy:value': ScalarInputVariable(variable_type='scalar', units='mm', parent_variable=None, value_range=[0.2000128411163704, 0.499960029319067], name='distgen:r_dist:sigma_xy:value', value=0.413, precision=None, default=0.413, is_constant=False),
 'distgen:t_dist:length:value': ScalarInputVariable(variable_type='scalar', units='ps', parent_variable=None, value_range=[3.0000935388576626, 11.999451344364768], name='distgen:t_dist:length:value', value=7.499772441611215, precision=None, default=7.499772441611215, is_constant=False),
 'distgen:total_charge:value': ScalarInputVariable(variable_type='scalar', units='pC', parent_variable=None, value_range=[250.0, 250.0], name='distgen:total_charge:value', value=250.0, precision=None, default=250.0, is_constant=True),
 'SOL1:solenoid_field_scale': ScalarInputVariable(variable_type='scalar', units='T', parent_variable=None, value_range=[0.15000051301097236, 0.29998391898360177], name='SOL1:solenoid_field_scale', value=0.17, p

In [9]:
output_variables

{'end_n_particle': ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_n_particle', value=9998.101, precision=None, default=None),
 'end_mean_gamma': ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_mean_gamma', value=147.09068, precision=None, default=None),
 'end_sigma_gamma': ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_sigma_gamma', value=0.54239374, precision=None, default=None),
 'end_mean_x': ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_mean_x', value=-1.0433723e-08, precision=None, default=None),
 'end_mean_y': ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_mean_y', value=-4.6587957e-08, precision=None, default=None),
 'end_norm_emit_x': ScalarOutputVariable(variable_type='scalar', units='m'

In [10]:
import torch
import matplotlib.pyplot as plt

from lume_model.utils import variables_from_yaml
from lume_model.torch import LUMEModule, PyTorchModel

In [11]:
input_transformer = torch.load("model/input_transformer.pt")
output_transformer = torch.load("model/output_transformer.pt")

In [12]:
input_variables, output_variables = variables_from_yaml(open("model/variables.yml"))

In [18]:
print(len(input_variables.keys()))
input_variables.keys()

16


dict_keys(['distgen:r_dist:sigma_xy:value', 'distgen:t_dist:length:value', 'distgen:total_charge:value', 'SOL1:solenoid_field_scale', 'CQ01:b1_gradient', 'SQ01:b1_gradient', 'L0A_scale:voltage', 'L0A_phase:dtheta0_deg', 'L0B_scale:voltage', 'L0B_phase:dtheta0_deg', 'QA01:b1_gradient', 'QA02:b1_gradient', 'QE01:b1_gradient', 'QE02:b1_gradient', 'QE03:b1_gradient', 'QE04:b1_gradient'])

In [14]:
# create lume model
lume_model = PyTorchModel(
    model_file="model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_transformer],
    output_transformers=[output_transformer],
)
lume_module = LUMEModule(
    model=lume_model,
    feature_order=lume_model.features,
    output_order=lume_model.outputs,
)
lume_module

LUMEModule(
  (base_model): Sequential(
    (0): Linear(in_features=16, out_features=100, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=100, out_features=200, bias=True)
    (3): ELU(alpha=1.0)
    (4): Dropout(p=0.05, inplace=False)
    (5): Linear(in_features=200, out_features=200, bias=True)
    (6): ELU(alpha=1.0)
    (7): Dropout(p=0.05, inplace=False)
    (8): Linear(in_features=200, out_features=300, bias=True)
    (9): ELU(alpha=1.0)
    (10): Dropout(p=0.05, inplace=False)
    (11): Linear(in_features=300, out_features=300, bias=True)
    (12): ELU(alpha=1.0)
    (13): Dropout(p=0.05, inplace=False)
    (14): Linear(in_features=300, out_features=200, bias=True)
    (15): ELU(alpha=1.0)
    (16): Dropout(p=0.05, inplace=False)
    (17): Linear(in_features=200, out_features=100, bias=True)
    (18): ELU(alpha=1.0)
    (19): Dropout(p=0.05, inplace=False)
    (20): Linear(in_features=100, out_features=100, bias=True)
    (21): ELU(alpha=1.0)
    (22): Linear(in_f

In [21]:
input_torch = {
                        "distgen:r_dist:sigma_xy:value": 0.4130,
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,

                        "L0B_scale:voltage": 0,
                        "L0B_phase:dtheta0_deg": 0,
                        "QA01:b1_gradient": 0,
                        "QA02:b1_gradient": 0,
                        "QE01:b1_gradient": 0,
                        "QE02:b1_gradient": 0,
                        "QE03:b1_gradient": 0,
                        "QE04:b1_gradient": 0                     
}

In [30]:
all_input_values = []
for key in input_torch:
        all_input_values.append(input_torch[key])
all_input_values = torch.Tensor([all_input_values])

In [31]:
all_input_values

tensor([[ 4.1300e-01,  2.5000e+02,  7.4998e+00,  1.7000e-01, -7.4000e-03,
         -7.4000e-03, -8.8997e+00,  7.0000e+07,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00]])

In [32]:
with torch.no_grad():
    predictions = lume_module(all_input_values)
predictions

tensor([-9.9168e+04,  2.2429e+05,  1.0245e+04,  6.5895e+01,  2.4669e+02],
       dtype=torch.float64)